# Processar Attributes

In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
df_input_path = '../../dados/X_testToronto.csv'

In [2]:
# Parameters
df_input_path = "../../dados/X_trainToronto.csv"
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/processar_attributes_data.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_processar_attributes.ipynb",
}


In [3]:
import numpy as np
import pandas as pd
import ast

In [4]:
def literal_eval(x):
    if pd.isna(x):
        return dict()
    raw_dict = ast.literal_eval(x)
    for key in raw_dict:
        raw_dict[key] = ast.literal_eval(raw_dict[key])
    return raw_dict
        
def proccess_df(df):
    #df['loc'] = df['loc'].apply(lambda x: ast.literal_eval(x))
    df['attributes'] = df['attributes'].apply(literal_eval)
    #df['hours'] = df['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else dict())

In [5]:
df = pd.read_csv(df_input_path)
proccess_df(df)
df_reg = df.copy()
df_reg = df_reg[['business_id', 'attributes']]
df_reg

,business_id,attributes
0,vHzWmPWHN4J1hRR3W3AMQg,"{'Ambience': {'romantic': False, 'intimate': F..."
1,15to24Q-otAHmto7FzsWRg,"{'BusinessParking': {'garage': False, 'street'..."
2,8aqKdf4G4AAir8k_Kdslvg,"{'ByAppointmentOnly': True, 'AcceptsInsurance'..."
3,uxU1vr5AhhkTQ83X0bpeyg,{'ByAppointmentOnly': False}
4,f702hTJoqdR34Jn23C7d1A,{'RestaurantsPriceRange2': 3}
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"{'Caters': True, 'BikeParking': False, 'WiFi':..."
17578,o2811xZ0ufRi7WcsqWiGzw,"{'RestaurantsPriceRange2': 2, 'BikeParking': F..."
17579,v9T9m_TbpLuaC2Kud5UbLA,"{'RestaurantsPriceRange2': 2, 'BikeParking': T..."
17580,bzGnAIe9OiBHGWJE-dZ_8g,"{'BusinessParking': {'garage': False, 'street'..."


### Transformar Attributes em dummy values

In [6]:
from collections import defaultdict

unique_attributes = defaultdict(set)
two_layer_attributtes = defaultdict(set)
attributes = df['attributes']
for item_dict in attributes:
    for key in item_dict:
        if type(item_dict[key]) is dict:
            for key2 in item_dict[key]:
                two_layer_attributtes[(key, key2)].add(item_dict[key][key2])
        else:
            unique_attributes[key].add(item_dict[key])

for attr1, attr2 in two_layer_attributtes:
    if attr1 in unique_attributes:
        unique_attributes.pop(attr1, None)

def dummy_attr(row):
    atributes = {}
    obj = row['attributes']
    if row['attributes'] == None:
        obj = dict()
    for key in unique_attributes:
        atributes[key] = None
        if key in obj:
            atributes[key] = obj[key]
            
    for attr1, attr2 in two_layer_attributtes:
        key = f'{attr1}.{attr2}'
        atributes[key] = None
        if attr1 in obj and type(obj[attr1]) is dict and attr2 in obj[attr1]:
            atributes[key] = obj[attr1][attr2]
    return atributes

df_attr = pd.DataFrame.from_dict(list(df_reg.apply(dummy_attr, axis=1)))
df_almost_dummie = pd.concat([df_reg, df_attr], axis=1)
new_columns = {at: 'attribute_' + at for at in df_attr.columns}
df_almost_dummie = df_almost_dummie.rename(columns=new_columns)

df_dummies = pd.get_dummies(df_almost_dummie[new_columns.values()].astype(str))
columns_to_drop = [x for x in list(df_dummies.columns) if ('_nan' in x or '_None' in x or '_none' in x)]
df_dummies = df_dummies.drop(columns_to_drop, axis=1)

df_reg = pd.concat([df_reg, df_dummies], axis=1)
df_reg

,business_id,attributes,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,attribute_WiFi_no,attribute_WiFi_paid,attribute_OutdoorSeating_False,attribute_OutdoorSeating_True,attribute_RestaurantsTakeOut_False,...,attribute_DietaryRestrictions.vegan_True,attribute_DietaryRestrictions.kosher_False,attribute_DietaryRestrictions.halal_False,attribute_DietaryRestrictions.halal_True,attribute_DietaryRestrictions.soy-free_False,attribute_DietaryRestrictions.vegetarian_False,attribute_DietaryRestrictions.vegetarian_True,attribute_HairSpecializesIn.straightperms_False,attribute_HairSpecializesIn.africanamerican_False,attribute_HairSpecializesIn.asian_False
0,vHzWmPWHN4J1hRR3W3AMQg,"{'Ambience': {'romantic': False, 'intimate': F...",True,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
1,15to24Q-otAHmto7FzsWRg,"{'BusinessParking': {'garage': False, 'street'...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,8aqKdf4G4AAir8k_Kdslvg,"{'ByAppointmentOnly': True, 'AcceptsInsurance'...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,uxU1vr5AhhkTQ83X0bpeyg,{'ByAppointmentOnly': False},False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,f702hTJoqdR34Jn23C7d1A,{'RestaurantsPriceRange2': 3},False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"{'Caters': True, 'BikeParking': False, 'WiFi':...",False,True,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
17578,o2811xZ0ufRi7WcsqWiGzw,"{'RestaurantsPriceRange2': 2, 'BikeParking': F...",False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
17579,v9T9m_TbpLuaC2Kud5UbLA,"{'RestaurantsPriceRange2': 2, 'BikeParking': T...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17580,bzGnAIe9OiBHGWJE-dZ_8g,"{'BusinessParking': {'garage': False, 'street'...",False,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
attributes_columns = [x for x in df_reg.columns if x.startswith('attribute_')]
df_reg[attributes_columns] = df_reg[attributes_columns].apply(lambda x: x.astype(float))

In [8]:
df_reg.to_parquet(product['data'])

## Teste manual

In [9]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['attributes'])
    display(df_reg[attributes_columns].iloc[[13]])
    display('----------------------------------')
    display(df_reg.iloc[542]['attributes'])
    display(df_reg[attributes_columns].iloc[[542]])
    display('----------------------------------')
    display(df_reg.iloc[5389]['attributes'])
    display(df_reg[attributes_columns].iloc[[5389]])

{'OutdoorSeating': True,
 'RestaurantsReservations': False,
 'RestaurantsDelivery': False,
 'RestaurantsPriceRange2': 2,
 'RestaurantsAttire': 'casual',
 'WiFi': 'no',
 'RestaurantsGoodForGroups': True,
 'Ambience': {'romantic': False,
  'intimate': False,
  'classy': False,
  'hipster': False,
  'divey': False,
  'touristy': False,
  'trendy': False,
  'upscale': False,
  'casual': True},
 'RestaurantsTakeOut': True,
 'Caters': False,
 'HasTV': True,
 'GoodForKids': True,
 'Alcohol': 'full_bar',
 'NoiseLevel': 'average',
 'BikeParking': True,
 'BusinessParking': {'garage': False,
  'street': True,
  'validated': False,
  'lot': False,
  'valet': False},
 'GoodForMeal': {'dessert': False,
  'latenight': False,
  'lunch': True,
  'dinner': False,
  'brunch': False,
  'breakfast': True}}

,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,attribute_WiFi_no,attribute_WiFi_paid,attribute_OutdoorSeating_False,attribute_OutdoorSeating_True,attribute_RestaurantsTakeOut_False,attribute_RestaurantsTakeOut_True,attribute_RestaurantsAttire_casual,attribute_RestaurantsAttire_dressy,attribute_RestaurantsAttire_formal,attribute_NoiseLevel_average,attribute_NoiseLevel_loud,attribute_NoiseLevel_quiet,attribute_NoiseLevel_very_loud,attribute_RestaurantsDelivery_False,attribute_RestaurantsDelivery_True,attribute_RestaurantsPriceRange2_1.0,attribute_RestaurantsPriceRange2_2.0,attribute_RestaurantsPriceRange2_3.0,attribute_RestaurantsPriceRange2_4.0,attribute_GoodForKids_False,attribute_GoodForKids_True,attribute_RestaurantsGoodForGroups_False,attribute_RestaurantsGoodForGroups_True,attribute_Alcohol_beer_and_wine,attribute_Alcohol_full_bar,attribute_BikeParking_False,attribute_BikeParking_True,attribute_HasTV_False,attribute_HasTV_True,attribute_RestaurantsReservations_False,attribute_RestaurantsReservations_True,attribute_ByAppointmentOnly_False,attribute_ByAppointmentOnly_True,attribute_AcceptsInsurance_False,attribute_AcceptsInsurance_True,attribute_RestaurantsTableService_False,attribute_RestaurantsTableService_True,attribute_WheelchairAccessible_False,attribute_WheelchairAccessible_True,attribute_DogsAllowed_False,attribute_DogsAllowed_True,attribute_BusinessAcceptsCreditCards_False,attribute_BusinessAcceptsCreditCards_True,attribute_Smoking_no,attribute_Smoking_outdoor,attribute_Smoking_yes,attribute_GoodForDancing_False,attribute_GoodForDancing_True,attribute_HappyHour_False,attribute_HappyHour_True,attribute_CoatCheck_False,attribute_CoatCheck_True,attribute_DriveThru_False,attribute_DriveThru_True,attribute_AgesAllowed_19plus,attribute_RestaurantsCounterService_True,attribute_BusinessAcceptsBitcoin_False,attribute_Ambience.romantic_False,attribute_Ambience.romantic_True,attribute_Ambience.intimate_False,attribute_Ambience.intimate_True,attribute_Ambience.classy_False,attribute_Ambience.classy_True,attribute_Ambience.hipster_False,attribute_Ambience.hipster_True,attribute_Ambience.divey_False,attribute_Ambience.touristy_False,attribute_Ambience.touristy_True,attribute_Ambience.trendy_False,attribute_Ambience.trendy_True,attribute_Ambience.upscale_False,attribute_Ambience.upscale_True,attribute_Ambience.casual_False,attribute_Ambience.casual_True,attribute_GoodForMeal.dessert_False,attribute_GoodForMeal.dessert_True,attribute_GoodForMeal.latenight_False,attribute_GoodForMeal.latenight_True,attribute_GoodForMeal.lunch_False,attribute_GoodForMeal.lunch_True,attribute_GoodForMeal.dinner_False,attribute_GoodForMeal.dinner_True,attribute_GoodForMeal.brunch_False,attribute_GoodForMeal.brunch_True,attribute_GoodForMeal.breakfast_False,attribute_GoodForMeal.breakfast_True,attribute_BusinessParking.garage_False,attribute_BusinessParking.garage_True,attribute_BusinessParking.street_False,attribute_BusinessParking.street_True,attribute_BusinessParking.validated_False,attribute_BusinessParking.validated_True,attribute_BusinessParking.lot_False,attribute_BusinessParking.lot_True,attribute_BusinessParking.valet_False,attribute_BusinessParking.valet_True,attribute_HairSpecializesIn.perms_False,attribute_HairSpecializesIn.perms_True,attribute_HairSpecializesIn.coloring_False,attribute_HairSpecializesIn.coloring_True,attribute_HairSpecializesIn.extensions_False,attribute_HairSpecializesIn.extensions_True,attribute_HairSpecializesIn.curly_False,attribute_HairSpecializesIn.curly_True,attribute_HairSpecializesIn.kids_False,attribute_HairSpecializesIn.kids_True,attribute_BestNights.monday_False,attribute_BestNights.monday_True,attribute_BestNights.tuesday_False,attribute_BestNights.tuesday_True,attribute_BestNights.friday_False,attribute_BestNights.friday_True,attribute_BestNights.wednesday_False,attribute_BestNights.wednesday_True,attribute_BestNights.thursday_False,attribute_BestNights.thursday_True,attribute_BestNights.sund

'----------------------------------'

{'RestaurantsTakeOut': True}

,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,attribute_WiFi_no,attribute_WiFi_paid,attribute_OutdoorSeating_False,attribute_OutdoorSeating_True,attribute_RestaurantsTakeOut_False,attribute_RestaurantsTakeOut_True,attribute_RestaurantsAttire_casual,attribute_RestaurantsAttire_dressy,attribute_RestaurantsAttire_formal,attribute_NoiseLevel_average,attribute_NoiseLevel_loud,attribute_NoiseLevel_quiet,attribute_NoiseLevel_very_loud,attribute_RestaurantsDelivery_False,attribute_RestaurantsDelivery_True,attribute_RestaurantsPriceRange2_1.0,attribute_RestaurantsPriceRange2_2.0,attribute_RestaurantsPriceRange2_3.0,attribute_RestaurantsPriceRange2_4.0,attribute_GoodForKids_False,attribute_GoodForKids_True,attribute_RestaurantsGoodForGroups_False,attribute_RestaurantsGoodForGroups_True,attribute_Alcohol_beer_and_wine,attribute_Alcohol_full_bar,attribute_BikeParking_False,attribute_BikeParking_True,attribute_HasTV_False,attribute_HasTV_True,attribute_RestaurantsReservations_False,attribute_RestaurantsReservations_True,attribute_ByAppointmentOnly_False,attribute_ByAppointmentOnly_True,attribute_AcceptsInsurance_False,attribute_AcceptsInsurance_True,attribute_RestaurantsTableService_False,attribute_RestaurantsTableService_True,attribute_WheelchairAccessible_False,attribute_WheelchairAccessible_True,attribute_DogsAllowed_False,attribute_DogsAllowed_True,attribute_BusinessAcceptsCreditCards_False,attribute_BusinessAcceptsCreditCards_True,attribute_Smoking_no,attribute_Smoking_outdoor,attribute_Smoking_yes,attribute_GoodForDancing_False,attribute_GoodForDancing_True,attribute_HappyHour_False,attribute_HappyHour_True,attribute_CoatCheck_False,attribute_CoatCheck_True,attribute_DriveThru_False,attribute_DriveThru_True,attribute_AgesAllowed_19plus,attribute_RestaurantsCounterService_True,attribute_BusinessAcceptsBitcoin_False,attribute_Ambience.romantic_False,attribute_Ambience.romantic_True,attribute_Ambience.intimate_False,attribute_Ambience.intimate_True,attribute_Ambience.classy_False,attribute_Ambience.classy_True,attribute_Ambience.hipster_False,attribute_Ambience.hipster_True,attribute_Ambience.divey_False,attribute_Ambience.touristy_False,attribute_Ambience.touristy_True,attribute_Ambience.trendy_False,attribute_Ambience.trendy_True,attribute_Ambience.upscale_False,attribute_Ambience.upscale_True,attribute_Ambience.casual_False,attribute_Ambience.casual_True,attribute_GoodForMeal.dessert_False,attribute_GoodForMeal.dessert_True,attribute_GoodForMeal.latenight_False,attribute_GoodForMeal.latenight_True,attribute_GoodForMeal.lunch_False,attribute_GoodForMeal.lunch_True,attribute_GoodForMeal.dinner_False,attribute_GoodForMeal.dinner_True,attribute_GoodForMeal.brunch_False,attribute_GoodForMeal.brunch_True,attribute_GoodForMeal.breakfast_False,attribute_GoodForMeal.breakfast_True,attribute_BusinessParking.garage_False,attribute_BusinessParking.garage_True,attribute_BusinessParking.street_False,attribute_BusinessParking.street_True,attribute_BusinessParking.validated_False,attribute_BusinessParking.validated_True,attribute_BusinessParking.lot_False,attribute_BusinessParking.lot_True,attribute_BusinessParking.valet_False,attribute_BusinessParking.valet_True,attribute_HairSpecializesIn.perms_False,attribute_HairSpecializesIn.perms_True,attribute_HairSpecializesIn.coloring_False,attribute_HairSpecializesIn.coloring_True,attribute_HairSpecializesIn.extensions_False,attribute_HairSpecializesIn.extensions_True,attribute_HairSpecializesIn.curly_False,attribute_HairSpecializesIn.curly_True,attribute_HairSpecializesIn.kids_False,attribute_HairSpecializesIn.kids_True,attribute_BestNights.monday_False,attribute_BestNights.monday_True,attribute_BestNights.tuesday_False,attribute_BestNights.tuesday_True,attribute_BestNights.friday_False,attribute_BestNights.friday_True,attribute_BestNights.wednesday_False,attribute_BestNights.wednesday_True,attribute_BestNights.thursday_False,attribute_BestNights.thursday_True,attribute_BestNights.sund

'----------------------------------'

{}

,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,attribute_WiFi_no,attribute_WiFi_paid,attribute_OutdoorSeating_False,attribute_OutdoorSeating_True,attribute_RestaurantsTakeOut_False,attribute_RestaurantsTakeOut_True,attribute_RestaurantsAttire_casual,attribute_RestaurantsAttire_dressy,attribute_RestaurantsAttire_formal,attribute_NoiseLevel_average,attribute_NoiseLevel_loud,attribute_NoiseLevel_quiet,attribute_NoiseLevel_very_loud,attribute_RestaurantsDelivery_False,attribute_RestaurantsDelivery_True,attribute_RestaurantsPriceRange2_1.0,attribute_RestaurantsPriceRange2_2.0,attribute_RestaurantsPriceRange2_3.0,attribute_RestaurantsPriceRange2_4.0,attribute_GoodForKids_False,attribute_GoodForKids_True,attribute_RestaurantsGoodForGroups_False,attribute_RestaurantsGoodForGroups_True,attribute_Alcohol_beer_and_wine,attribute_Alcohol_full_bar,attribute_BikeParking_False,attribute_BikeParking_True,attribute_HasTV_False,attribute_HasTV_True,attribute_RestaurantsReservations_False,attribute_RestaurantsReservations_True,attribute_ByAppointmentOnly_False,attribute_ByAppointmentOnly_True,attribute_AcceptsInsurance_False,attribute_AcceptsInsurance_True,attribute_RestaurantsTableService_False,attribute_RestaurantsTableService_True,attribute_WheelchairAccessible_False,attribute_WheelchairAccessible_True,attribute_DogsAllowed_False,attribute_DogsAllowed_True,attribute_BusinessAcceptsCreditCards_False,attribute_BusinessAcceptsCreditCards_True,attribute_Smoking_no,attribute_Smoking_outdoor,attribute_Smoking_yes,attribute_GoodForDancing_False,attribute_GoodForDancing_True,attribute_HappyHour_False,attribute_HappyHour_True,attribute_CoatCheck_False,attribute_CoatCheck_True,attribute_DriveThru_False,attribute_DriveThru_True,attribute_AgesAllowed_19plus,attribute_RestaurantsCounterService_True,attribute_BusinessAcceptsBitcoin_False,attribute_Ambience.romantic_False,attribute_Ambience.romantic_True,attribute_Ambience.intimate_False,attribute_Ambience.intimate_True,attribute_Ambience.classy_False,attribute_Ambience.classy_True,attribute_Ambience.hipster_False,attribute_Ambience.hipster_True,attribute_Ambience.divey_False,attribute_Ambience.touristy_False,attribute_Ambience.touristy_True,attribute_Ambience.trendy_False,attribute_Ambience.trendy_True,attribute_Ambience.upscale_False,attribute_Ambience.upscale_True,attribute_Ambience.casual_False,attribute_Ambience.casual_True,attribute_GoodForMeal.dessert_False,attribute_GoodForMeal.dessert_True,attribute_GoodForMeal.latenight_False,attribute_GoodForMeal.latenight_True,attribute_GoodForMeal.lunch_False,attribute_GoodForMeal.lunch_True,attribute_GoodForMeal.dinner_False,attribute_GoodForMeal.dinner_True,attribute_GoodForMeal.brunch_False,attribute_GoodForMeal.brunch_True,attribute_GoodForMeal.breakfast_False,attribute_GoodForMeal.breakfast_True,attribute_BusinessParking.garage_False,attribute_BusinessParking.garage_True,attribute_BusinessParking.street_False,attribute_BusinessParking.street_True,attribute_BusinessParking.validated_False,attribute_BusinessParking.validated_True,attribute_BusinessParking.lot_False,attribute_BusinessParking.lot_True,attribute_BusinessParking.valet_False,attribute_BusinessParking.valet_True,attribute_HairSpecializesIn.perms_False,attribute_HairSpecializesIn.perms_True,attribute_HairSpecializesIn.coloring_False,attribute_HairSpecializesIn.coloring_True,attribute_HairSpecializesIn.extensions_False,attribute_HairSpecializesIn.extensions_True,attribute_HairSpecializesIn.curly_False,attribute_HairSpecializesIn.curly_True,attribute_HairSpecializesIn.kids_False,attribute_HairSpecializesIn.kids_True,attribute_BestNights.monday_False,attribute_BestNights.monday_True,attribute_BestNights.tuesday_False,attribute_BestNights.tuesday_True,attribute_BestNights.friday_False,attribute_BestNights.friday_True,attribute_BestNights.wednesday_False,attribute_BestNights.wednesday_True,attribute_BestNights.thursday_False,attribute_BestNights.thursday_True,attribute_BestNights.sund